# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)


In [1]:
import mlx.core as mx
import mlx.nn as nn

net = nn.Sequential(nn.Linear(4, 8),
                    nn.ReLU(),
                    nn.Linear(8, 1))

X = mx.random.uniform(shape=(2, 4))
net(X).shape

(2, 1)

## [**参数访问**]

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。


In [2]:
net.layers[2].parameters()

{'weight': array([[-0.347928, 0.0240712, -0.230242, ..., 0.277493, 0.248133, -0.215919]], dtype=float32),
 'bias': array([0.306495], dtype=float32)}

输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### [**目标参数**]

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [3]:
type(net.layers[2].bias), net.layers[2].bias

(mlx.core.array, array([0.306495], dtype=float32))

### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [4]:
name, layers = net.named_modules()[::-1][0]
for param in layers.parameters():
    print(name, param, layers[param].shape)

print(*[(name, param, layers[param].shape) for name, layers in net.named_modules()[1:][::-1] for param in layers.parameters()])

layers.0 weight (8, 4)
layers.0 bias (8,)
('layers.0', 'weight', (8, 4)) ('layers.0', 'bias', (8,)) ('layers.2', 'weight', (1, 8)) ('layers.2', 'bias', (1,))


这为我们提供了另一种访问网络参数的方式，如下所示。


In [5]:
net['layers'][2]['bias']

array([0.306495], dtype=float32)

### [**从嵌套块收集参数**]

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [6]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = []
    for i in range(4):
        # 在这里嵌套
        net.append(block1())
    return net

rgnet = nn.Sequential(*block2(), nn.Linear(4, 1))
rgnet(X)

array([[-0.479976],
       [-0.479971]], dtype=float32)

[**设计了网络后，我们看看它是如何工作的。**]


In [7]:
print(rgnet)

Sequential(
  (layers.0): Sequential(
    (layers.0): Linear(input_dims=4, output_dims=8, bias=True)
    (layers.1): ReLU()
    (layers.2): Linear(input_dims=8, output_dims=4, bias=True)
    (layers.3): ReLU()
  )
  (layers.1): Sequential(
    (layers.0): Linear(input_dims=4, output_dims=8, bias=True)
    (layers.1): ReLU()
    (layers.2): Linear(input_dims=8, output_dims=4, bias=True)
    (layers.3): ReLU()
  )
  (layers.2): Sequential(
    (layers.0): Linear(input_dims=4, output_dims=8, bias=True)
    (layers.1): ReLU()
    (layers.2): Linear(input_dims=8, output_dims=4, bias=True)
    (layers.3): ReLU()
  )
  (layers.3): Sequential(
    (layers.0): Linear(input_dims=4, output_dims=8, bias=True)
    (layers.1): ReLU()
    (layers.2): Linear(input_dims=8, output_dims=4, bias=True)
    (layers.3): ReLU()
  )
  (layers.4): Linear(input_dims=4, output_dims=1, bias=True)
)


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。


In [8]:
rgnet.layers[1].layers[0].bias

array([-0.141924, 0.245702, -0.203424, ..., -0.473626, 0.289716, 0.353636], dtype=float32)

## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在 :numref:`sec_numerical_stability`中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。


默认情况下，MLX会根据一个范围均匀地初始化权重和偏置矩阵，这个范围是根据输入和输出维度计算出的。MLX的nn.init模块提供了多种预置初始化方法。


### [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


In [9]:
# def init_normal(array):
#     weight_fn = nn.init.normal(mean=0.0, std=0.01)
#     bias_fn = nn.init.constant(0.0)
#     if array.ndim > 1:
#         array = weight_fn(array)
#     else:
#         array = bias_fn(array)
#     return array

# for module in net.modules():
#     if isinstance(module, nn.Linear):
#         module.update(mlx.utils.tree_map(lambda x: init_normal(x), module.parameters()))

weight_fn = nn.init.normal(mean=0.0, std=0.01)
bias_fn = nn.init.constant(0.0)
for layer in net.layers:
    if type(layer) == nn.Linear:
        layer.weight = weight_fn(layer.weight)
        layer.bias = bias_fn(layer.bias)

net.layers[0].weight[0], net.layers[0].bias[0]

(array([-0.00447087, -0.0227117, 0.016953, -0.00380947], dtype=float32),
 array(0, dtype=float32))

我们还可以将所有参数初始化为给定的常数，比如初始化为1。


In [10]:
# def init_constant(array):
#     weight_fn = nn.init.constant(1.0)
#     bias_fn = nn.init.constant(0.0)
#     if array.ndim > 1:
#         array = weight_fn(array)
#     else:
#         array = bias_fn(array)
#     return array

# for module in net.modules():
#     if isinstance(module, nn.Linear):
#         module.update(mlx.utils.tree_map(lambda x: init_constant(x), module.parameters()))

weight_fn = nn.init.constant(1.0)
bias_fn = nn.init.constant(0.0)
for layer in net.layers:
    if type(layer) == nn.Linear:
        layer.weight = weight_fn(layer.weight)
        layer.bias = bias_fn(layer.bias)

net.layers[0].weight[0], net.layers[0].bias[0]

(array([1, 1, 1, 1], dtype=float32), array(0, dtype=float32))

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
然后将第三个神经网络层初始化为常量值42。


In [11]:
weight_fn = nn.init.constant(1.0)
bias_fn = nn.init.constant(0.0)
for layer in net.layers:
    if type(layer) == nn.Linear:
        layer.weight = weight_fn(layer.weight)
        layer.bias = bias_fn(layer.bias)

net.layers[0].weight[0], net.layers[0].bias[0]

(array([1, 1, 1, 1], dtype=float32), array(0, dtype=float32))

In [12]:
# def init_xavier(array):
#     if array.ndim > 1: # bias
#         weight_fn = nn.init.glorot_uniform()
#         array = weight_fn(array)
#     return array

# def init_42(array):
#     if array.ndim > 1: # bias
#         weight_fn = nn.init.constant(42.0)
#         array = weight_fn(array)
#     return array

# for i, module in enumerate(net.modules()[1:][::-1]):
#     if i == 0:
#         module.update(mlx.utils.tree_map(lambda x: init_xavier(x), module.parameters()))
#     if i == 2:
#         module.update(mlx.utils.tree_map(lambda x: init_42(x), module.parameters()))

uniform_fn = nn.init.glorot_uniform()
const_fn = nn.init.constant(42.0)
net.layers[0].weight = uniform_fn(net.layers[0].weight)
net.layers[2].weight = const_fn(net.layers[2].weight)
print(net.layers[0].weight[0])
print(net.layers[2].weight)

array([-0.238554, 0.592675, 0.250875, 0.421302], dtype=float32)
array([[42, 42, 42, ..., 42, 42, 42]], dtype=float32)


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


In [13]:
for name, layer in net.named_modules()[::-1]:
    if type(layer) == nn.Linear:
        for param in layer.parameters():
            print("Init", param, layer[param].shape)
            break
        weight_fn = nn.init.uniform(low=-10, high=10)
        layer.weight = weight_fn(layer.weight)
        layer.weight *= mx.abs(layer.weight) >= 5

net.layers[0].weight[:2]

Init weight (8, 4)
Init weight (1, 8)


array([[6.26438, 7.84126, 7.06211, -9.43468],
       [0, 0, -9.56239, 9.42035]], dtype=float32)

注意，我们始终可以直接设置参数。


In [14]:
net.layers[0].weight[:] += 1
net.layers[0].weight[0, 0] = 42
net.layers[0].weight[0]

array([42, 8.84126, 8.06211, -8.43468], dtype=float32)

## [**参数绑定**]

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [15]:
# 定义模型
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

net(X)
# 检查参数是否相同
print(net.layers[2].weight[0] == net.layers[2].weight[0])
net.layers[2].weight[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net.layers[2].weight[0]  == net.layers[4].weight[0])

array([True, True, True, ..., True, True, True], dtype=bool)
array([True, True, True, ..., True, True, True], dtype=bool)


## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用 :numref:`sec_model_construction` 中定义的`FancyMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？
